The version below verifies if the first letter of the code + the four digits are already in the final list, if not, it doesn't add. (best so far)

In [18]:
import PyPDF2
import re

instrument_codes = ['A', 'C', 'F', 'P', 'S', 'L', 'H', 'X']
black_list = ['AL', 'PA', 'AD', 'HI', 'LLL', 'LLL', 'LO', 'LHH', 'HLL']

filepath = "/Users/marcelosouza/Documents/Kraton/PIDs/"

pdf_file = open(filepath+'CarbonAbsorber_PID_20230317_1.pdf', 'rb') 

pdf_reader = PyPDF2.PdfReader(pdf_file)

instrument_texts = []

for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    page_text = page.extract_text()

    page_instruments = []
    for line in page_text.split('\n'):
        for code in instrument_codes:
            pattern = r'((?:' + code + r'\w{1,2})\d{4}|\d{4}(?:' + code + r'\w{1,2}))'
            matches = re.findall(pattern, line)

            for match in matches:
                instrument_code_match = re.search(r'\D{2,3}', match)
                num_match = re.search(r'\d{4}', match)
                
                if instrument_code_match and num_match:
                    instrument_code = instrument_code_match.group()
                    num = num_match.group()

                    # Check if the instrument code is not in the black_list
                    if instrument_code not in black_list and int(num[0]) >= 5:
                        # Check if the first letter and the 4 digits are not already in the final list
                        should_add = True
                        for existing_code in page_instruments:
                            existing_letter = existing_code.split('-')[0][0]
                            existing_digits = existing_code.split('-')[1]
                            if instrument_code[0] == existing_letter and num == existing_digits:
                                should_add = False
                                break

                        if should_add:
                            page_instruments.append(f"{instrument_code}-{num}")

    instrument_texts.append(page_instruments)

pdf_file.close()

print('Instrument texts:', instrument_texts)


Instrument texts: [['ACE-5096', 'PSV-5720', 'PI-5719', 'SO-5723', 'XO-5723', 'FD-5723', 'AHS-5723', 'PI-5722', 'PI-5758', 'PDI-5759', 'PSV-5828', 'AI-5760', 'AE-5889', 'HHH-5721', 'FC-5721', 'PI-5209', 'PDI-5208', 'FE-5762', 'PI-5817', 'HVI-5198', 'PIN-5801']]


possible solution, using CV to check circles on P&ID, then extracting text and comparing to the codes above. --> still needs work: it is detecting the circles BUT it is not extracting the text correctly.

In [8]:
import cv2
import pytesseract
import numpy as np
from PIL import Image
import re
from pdf2image import convert_from_path

# Convert PDF to images
def pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    return images

def detect_circles(image):
    # Resize the image to zoom in
    scale_percent = 200  # percentage of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20, param1=250, param2=100, minRadius=1, maxRadius=0)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # Filter out non-circular shapes
        filtered_circles = []
        for circle in circles[0, :]:
            x, y, r = circle
            mask = np.zeros_like(gray)
            cv2.circle(mask, (x, y), r, 255, -1)
            
            # Adjust the following line based on the OpenCV version
            contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            contours = contours[0] if len(contours) == 2 else contours[1]

            # Iterate over contours and check circularity
            for contour in contours:
                area = cv2.contourArea(contour)
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter)

                # Keep only shapes with circularity close to 1
                if 0.8 < circularity < 1.2:
                    filtered_circles.append(circle)
                    break

        return np.array([filtered_circles])
    else:
        return None

def extract_text_from_circles(image, circles):
    texts = []
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            x, y, r = circle
            x, y, r = np.int64(x), np.int64(y), np.int64(r)  # Cast variables to int64
            x_min, x_max = max(x - r, 0), min(x + r, image.shape[1])
            y_min, y_max = max(y - r, 0), min(y + r, image.shape[0])
            cropped_image = image[y_min:y_max, x_min:x_max]
            
            # Show the image
            # cv2.imshow("Cropped", cropped_image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows() 

            # Check if the cropped image has a valid shape
            if cropped_image.shape[0] > 0 and cropped_image.shape[1] > 0:
                # Preprocess the cropped image
                gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
                _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # OCR configurations
                custom_config = '--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-'

                text = pytesseract.image_to_string(binary, config=custom_config).strip()

                # Post-process the text
                text_lines = text.split('\n')

                if len(text_lines) == 2:
                    text = f"{text_lines[0]}-{text_lines[1]}"
                    texts.append(text)
    return texts

# Main function
def main():
    filepath = "/Users/marcelosouza/Documents/Kraton/PIDs/"
    # pdf_path = filepath+'CarbonAbsorber_PID_20230317_2.pdf'
    pdf_path = filepath+'test_pid.pdf'
    images = pdf_to_images(pdf_path)
    all_valid_texts = []

    for image in images:
        image = np.array(image)
        circles = detect_circles(image)
        texts = extract_text_from_circles(image, circles)
        all_valid_texts.extend(texts)

    print('Valid instrument texts:', all_valid_texts)
    print('Valid instrument texts:', circles)

if __name__ == "__main__":
    main()

Valid instrument texts: []
Valid instrument texts: [[[ 616 1622   99]
  [ 912 1620   99]
  [1108  804   99]]]


Detecting circles in a dummy image

In [1]:
import cv2
import numpy as np

def detect_circles(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20, param1=250, param2=140, minRadius=1, maxRadius=0)
    return circles

def draw_labeled_circles(image, circles):
    labeled_image = image.copy()
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for idx, circle in enumerate(circles[0, :]):
            x, y, r = circle
            cv2.circle(labeled_image, (x, y), r, (0, 255, 0), 2)
            cv2.putText(labeled_image, str(idx+1), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    return labeled_image

# Load your image
filepath = "/Users/marcelosouza/Documents/Kraton/PIDs/"
image_path = filepath+'test_pid.jpg'
image = cv2.imread(image_path)

# Detect circles
circles = detect_circles(image)

# Draw labeled circles on the image
labeled_image = draw_labeled_circles(image, circles)

# Show the labeled image
cv2.imshow("Labeled Circles", labeled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)


-1

: 